### API Calls from OpenFEMA

In [37]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json

In [18]:
# Load cyber security breach data set into Pandas
cyber_data = pd.read_csv("../source_data/cybersecurity_data.csv")
cyber_data.head()

,Unnamed: 0,Number,Name_of_Covered_Entity,State,Business_Associate_Involved,Individuals_Affected,Date_of_Breach,Type_of_Breach,Location_of_Breached_Information,Date_Posted_or_Updated,Summary,breach_start,breach_end,year
0,141,140,UNCG Speech and Hearing Center,NC,NaN,2300,1/1/1997,Hacking/IT Incident,Desktop Computer,1/23/2014,NaN,1/1/1997,NaN,1997
1,1030,1029,UMass Memorial Medical Center,MA,NaN,2387,5/6/2002,Unauthorized Access/Disclosure,"Electronic Medical Record, Paper",5/27/2014,NaN,5/6/2002,NaN,2002
2,212,211,Riverside Mercy Hospital and Ohio/Mercy Diagno...,OH,NaN,1000,3/29/2003,Improper Disposal,Paper,1/23/2014,NaN,3/29/2003,NaN,2003
3,272,271,SW General Inc,AZ,NaN,566,5/1/2004,Theft,Paper,1/23/2014,NaN,5/1/2004,NaN,2004
4,223,222,OhioHealth Corporation dba Grant Medical Center,OH,NaN,501,1/1/2008,Theft,"Laptop, Desktop Computer",1/23/2014,NaN,1/1/2008,NaN,2008


In [19]:
# Count the number of rows of data in data set 
len(cyber_data)

1055

In [20]:
# Extract the relevent columns from the inital data set
cyber_columns = cyber_data[["Number", "State", "year", "Date_of_Breach"]]
cyber_columns.head()

,Number,State,year,Date_of_Breach
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [21]:
# Rename columns to better describe the data set
cyber_columns_renamed = cyber_columns.rename(columns={"Number": "Breach ID", "year": "Year","Date_of_Breach": "Full Date"})
cyber_columns_renamed.head()

,Breach ID,State,Year,Full Date
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [22]:
# Remove rows that contain data from states outside the continental US (Alaska, Hawaii, and Puerto Rico)
# Note this data set includes DC, need to find out if wildfire data contains DC or not, kept it in cyber dataframe for now

cyber_data_clean = cyber_columns_renamed.loc[ (cyber_columns_renamed["State"] != "AK") & (cyber_columns_renamed["State"] != "HI") & (cyber_columns_renamed["State"] != "PR") ]
cyber_data_clean.head()

,Breach ID,State,Year,Full Date
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [23]:
# Count number of rows of cleaned data to make sure irrelevant rows were removed
len(cyber_data_clean)

1018

In [26]:
# Write the cleaned data to a new CSV file called 'cyber_data_clean.csv'
cyber_data_clean.to_csv("../source_data/cyber_data_clean.csv")

In [39]:
#Pulling in states from data breach csv
file_path = "../source_data/cyber_data_clean.csv"
clean_df = pd.read_csv(file_path)
state_list = clean_df['State'].unique()

In [40]:
#Base URL for OpenFEMA database
base_url = f'https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$orderby=incidentBeginDate&$select=incidentType,state,incidentBeginDate&$inlinecount=allpages&$skip='

In [41]:
#List where disaster summaries will be stored
disas_summs = []
#Range of data limited between years 1997 and 2014
#Since the range is hard stuck between these limits, hard coding is justified
start = 15940 #Page starts with first disaster in 1997 
end = 43387 #Page ends with last disaster in 2014

In [46]:
while start < 45000: #We want the loop to end after finishing the 42,940th page 
                     #43387 - 15940 = 27447, and 15940 + 27000 = 42940  
    #Combined URL
    comb_url = base_url + str(start)
    #Grab response from URL
    response = requests.get(comb_url).json()
    #Grab list of disaster summaries from response
    summ_list = response['DisasterDeclarationsSummaries']
    #Loops through summ_list 
    for i in range(len(summ_list)):
        #Grab the correct state from response 
        given_state = summ_list[i]['state']
        #Check if the given state is in the state list (not AK, HI, or PR)
        if given_state in state_list:
            #If not one of these three, then check if the incident type contains the word Fire
            #Puts the string in incidentType to lower case before comparing strings to avoid case error
            if 'fire' in summ_list[i]['incidentType'].lower():
                #Appends following dictionary to disas_summs
                disas_summs.append(summ_list[i])
    #After appending, start is increased by 1000 to avoid summary overlap (values repeating in append) 
    start += 1000

In [47]:
#After the first 27 loops in the while loop, the last page after this must be run to get
#the remaining 2014 summaries
#This will also accidently pick up summaries from after 2014 that need to be deleted
#pg_num = 42951
#comb_url = base_url + str(pg_num)
#response = requests.get(comb_url).json()
#summ_list = response['DisasterDeclarationsSummaries'] 

In [48]:
#Loops through summ_list 
#for i in range(len(summ_list)):
#    #Grab the correct state from response 
#    given_state = summ_list[i]['state']
#    #Check if the given state is in the state list (not AK, HI, or PR)
#    if given_state in state_list:
        #If not one of these three, then check if the incident type contains the word Fire
        #Puts the string in incidentType to lower case before comparing strings to avoid case error
#        if 'fire' in summ_list[i]['incidentType'].lower():
            #Appends following dictionary to disas_summs
#            disas_summs.append(summ_list[i])

In [49]:
pprint(disas_summs)

[{'id': '5f7b2c0731a8c6681cfd6158',
  'incidentBeginDate': '1997-07-21T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'WA'},
 {'id': '5f7b2c0731a8c6681cfd6175',
  'incidentBeginDate': '1997-08-14T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'WA'},
 {'id': '5f7b2c0731a8c6681cfd61a2',
  'incidentBeginDate': '1997-09-26T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'WA'},
 {'id': '5f7b2c0731a8c6681cfd616e',
  'incidentBeginDate': '1998-05-10T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'TX'},
 {'id': '5f7b2c0731a8c6681cfd6193',
  'incidentBeginDate': '1998-05-20T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'TX'},
 {'id': '5f7b2bf431a8c6681cfc3cfb',
  'incidentBeginDate': '1998-05-25T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'FL'},
 {'id': '5f7b2bf431a8c6681cfc3d04',
  'incidentBeginDate': '1998-05-25T00:00:00.000Z',
  'incidentType': 'Fire',
  'state': 'FL'},
 {'id': '5f7b2bf431a8c6681cfc3d08',
  'incidentBeginDate': '1998-05-25T00:00:00.000